# 2.8 通过微调提升模型的准确度与效率


模型微调对硬件性能要求较高，因此我们使用PAI的交互式建模（DSW）重新创建一个包含GPU的实例，让我们可以更快地完成微调任务。

请在另一个窗口打开课程章节“1_0_计算环境准备”的内容作为参考，环境准备操作可以参考[计算环境准备](https://edu.aliyun.com/course/3130200/lesson/343181920)中的**步骤一：创建PAI DSW实例**，不同之处在于：
1. 你需要给新的实例命名，比如acp_gpu，你需要确保这个名字不与运行2.7之前课程的实例重名；
2. **资源规格**选择免费试用页签中的**ecs.gn7i-c8g1.2xlarge**，不用担心，你领取到的免费额度足够支持你完成2.7与2.8的课程；
2. **镜像**选择**modelscope:1.18.0-pytorch2.3.0-gpu-py310-cu121-ubuntu22.04**（需要将“镜像配置”->”芯片类型“切换为GPU）。

    <img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/ZWGl0wWZx7kDO34Y/img/90dbff43-4e52-4f06-9bcc-c7395898d893.png" alt="" width="400px">

4. 当实例状态为运行中时，重复[计算环境准备](https://edu.aliyun.com/course/3130200/lesson/343181920)中的**步骤二**，获取大模型ACP课程的代码。

6. 当完成以上步骤后，请在你新创建的GPU实例中再打开本章节内容。

> 如果你在本地没有GPU环境，不建议你在本地学习本课程，代码的运行可能会失败。

## 🚄 前言

在前面的学习中，我们已经搭建了一个具有复杂功能的问答机器人了。你可能已经发现，随着流程变得复杂，你的应用响应也在变慢。因为目前每一个处理节点都是大模型。

在实际生产落地过程中，很多企业会尝试将一些任务单一的节点（如执行NL2SQL的大模型），替换成参数量更小的模型，并对其进行微调，让它在特定的领域的表现可以对标更大参数规模的模型，来换取更快的响应速度，与更低的成本开销。

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/92c133a5-3ba1-4f97-b8f5-2d98de7843e2.png" alt="NL2SQL示例" width="600px">

## 🍁 课程目标

学完本课程后，你将学习到：

*   什么是模型微调
    
*   如何通过swift框架对qwen2.5模型进行微调
        


## 📖 课程目录

- 1.&nbsp; 为什么需要对模型微调
- 2.&nbsp; 如何对模型微调

## 0. 环境准备

In [ ]:
# 由于更换实例，你需要重新安装依赖
! pip install -r requirements.txt

In [ ]:
# 除了之前安装的依赖外，你还需要安装以下依赖
! pip install accelerate==1.0.1 \
             rouge-score==0.1.2 \
             nltk==3.9.1 \
             ms-swift[llm]==2.4.2.post2 \
             evalscope==0.5.5rc1

In [2]:
# 配置环境变量
# 百炼 API Key获取链接：https://bailian.console.aliyun.com/?apiKey=1#/api-key
import os
from config.load_key import load_key
load_key()
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：*****


## 💻 1. 为什么需要对模型微调

在2.5节我们构造了一个查询员工信息的工具，用到了NL2SQL的方法。

答疑机器人使用了qwen-plus作为基础模型，尽管效果十分出色，但为了降低成本，以及获取更快的响应速度，你决定使用百炼免费提供（截止文档发布前）的qwen2.5-1.5b-instruct 模型推理API来作为NL2SQL的节点。于是你把model_name从`qwen-plus`更改为`qwen2.5-1.5b-instruct`。

In [ ]:
# 导入依赖
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

# 定义一个员工查询函数
def query_employee_info(query):
    llm = DashScope(model_name="qwen2.5-1.5b-instruct")
    messages = [
        ChatMessage(role=MessageRole.SYSTEM, content='''你是一个擅长写SQL的人，你有一个表叫employees，记录公司的员工信息，这个表有department（部门）、name（姓名）、HR三个字段。
    department中有教育部门、后勤部门、技术部门。
    你需要根据用户输入生成sql语句进行查询,你一定不能生成sql语句之外的内容，也不要把```sql```这个信息加上。'''),
        ChatMessage(role=MessageRole.USER, content=query)
    ]
    SQL_output = llm.chat(messages).message.content
    print(f'SQL语句为：{SQL_output}')
    if SQL_output == "SELECT COUNT(*) FROM employees WHERE department = '教育部门'":
        return "教育部门共有66名员工。"
    if SQL_output == "SELECT HR FROM employees WHERE name = '张三'":
        return "张三的HR是李四。"
    if SQL_output == "SELECT department FROM employees WHERE name = '王五'":
        return "王五的部门是后勤部。"
    else:
        return "抱歉，我暂时无法回答您的问题。"

query_employee_info("教育部门有几个人")

可以看到，更改模型后，输出的SQL语句中多了一个“；”，并且加上了```前后缀（提示词已明确写明不要添加），造成了查询失败的情况。

你可以通过模型微调的方法，降低输出错误SQL语句的概率。
>为了演示方便，此处使用了一个简单的问题作为示例。实际场景中，你依然可以通过优化提示词、增加后处理模块来提升NL2SQL的查询成功率。

##  🧠 2. 如何对模型微调
“微”字是对比预训练阶段，对训练数据的规模、算力与训练时间的要求要小很多；“调”则是调整模型的参数，这也是**模型训练**的本质。

>你可以将大模型理解为由多个矩阵组合在一起的系统，而矩阵内部的数值则是模型训练要调整的目标。

大模型的预训练一般都会使用海量的训练样本、成本高昂的算力与较长的时间来完成，微调则是在预训练模型的基础上进行特定任务上的调优，就像站在了巨人的肩膀上，因此所需的数据规模、算力要求与训练时间一般都远小于预训练阶段。

从调整目标参数的规模维度，可以将模型微调分为两大类，分别是**全参微调**与**高效微调**。

全参微调会调整模型的所有参数，因此对于算力要求较高；而高效微调旨在调整较少参数的基础上，达成与全参微调接近的效果。一般来说，高效微调的效果会略差于全参微调，但是可以节省较多的计算资源。

我们在本教程中向你介绍如何通过高效微调的方法来增强大模型NL2SQL的能力。

### 2.1. LoRA微调介绍
LoRA微调是一种效果与全参微调接近，并且最通用的高效微调方法之一。它通过引入一组低秩矩阵来调整预训练模型的权重，而不是直接更新所有参数。这种方法能够在不显著增加参数量的情况下，有效地对模型进行微调，特别适合于资源有限或需要快速迭代的场景。LoRA的核心思想是，对于每一个要调整的权重矩阵`D`，它不是直接修改`D`，而是添加一个低秩矩阵`A * B`（其中`A`和`B`的维度远小于`D`），这样既保留了原始模型的泛化能力，又实现了对特定任务的适应性增强。

假设我们有一个维度为3 * 3的`D`矩阵需要微调，LoRA并不直接对`D`矩阵进行参数更新，而是对维度为3 * 1的`A`矩阵和维度为1 * 3的`B`矩阵进行更新，最后用`A`矩阵乘`B`矩阵得到一个3 * 3的`C`矩阵，再将`C`矩阵叠加到`D`矩阵上，达到对`D`矩阵的更新。在这个例子中，我们更新的矩阵共6个参数（3个来自`A`矩阵，3个来自`B`矩阵），比直接更新`D`矩阵的9个参数少了3个。你也许觉得参数量没有少太多，但大模型至少有1亿的参数，如果我们要微调的矩阵维度是10000 * 10000，那么通过LoRA方法能够少调整的参数有：10000 * 10000-2 * 10000=9980000 个，需要更新的参数量减少了非常多。

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/b2e2caf3-e8da-4ac8-b57d-abbb05f932c4.png" alt="lora原理" width="800px">

如果你对LoRA的更多细节感兴趣，可以阅读：

[_LoRA: Low-Rank Adaptation of Large Language Models_](https://arxiv.org/abs/2106.09685)

LoRA有许多优点:
1. 微调得到的结果与预训练模型可以完全分离，因此不会破坏预训练模型的结构；
<br><i style="color: #999"> 因为微调得到的模型通过附加在预训练模型上达到改变参数的效果，在不需要LoRA时取消附加即可。 </i><br>
2. 实现方便，易于开发者快速微调和集成。
3. 即使使用了较少的参数量，也可以达到与全参微调类似的效果。

### 2.2. 微调数据集的构建

在大模型的对话场景中，一般会包含三个角色：`system`、`user`与`assistant`，每个角色都有对应的消息，就像聊天软件的头像与对话一样。

`system`角色消息用于指定大模型的目标或角色，也可以用于提供背景信息；`user`角色消息用于接收用户的提问；`assistant`角色消息表示大模型的输出。

我们微调的目的是让模型输出正确的SQL语句，因此可以将数据库的信息（如表名、字段名等）放入`system`角色消息中提供背景信息；将用户的提问放入`user`角色消息；将正确的SQL语句（可能需要人工标注）放入`assistant`角色消息来为大模型的输出提供正确指引。

我们将这三个角色的消息放在一个JSON里。下边的json是微调数据集的其中一个样本，微调数据集文件为`jsonl`格式，会包含多个符合下文格式的JSON。

```json
{
    "messages": [
        {
            "role": "system",
            "content": "#数据库的信息"
        },
        {
            "role": "user",
            "content": "#用户要求输出SQL的提问"
        },
        {
            "role": "assistant",
            "content": "#正确的SQL语句"
        }
    ]
}
```

我们已经准备好了一份NL2SQL的数据集，在`resources/2_4/data.zip`文件，你需要运行以下命令进行解压：

In [ ]:
!unzip -u resources/2_4/data.zip -d resources/2_4

解压后的data文件夹在resources/2_4中，打开data文件夹可以看到

<img src="https://alidocs.oss-cn-zhangjiakou.aliyuncs.com/res/oJGq765B3YDrnAKe/img/67e3aae7-7340-416e-a60c-f043ffc1fe53.png" alt="解压包文件" width="300px">

其中train.jsonl为训练集文件，test.jsonl为测试集文件。

我们在这里为你介绍在模型训练时的数据分类：

*   **训练集**：用于在**训练阶段**进行模型参数的训练。
    
*   **验证集**：用于在**训练阶段**评估模型训练的效果，从而调整模型的超参数（如学习率、退火策略等）和监控过拟合情况。
    
*   **测试集**：用于在**训练阶段完成后**评估模型训练的效果。本课程提供的实验数据中，测试集涉及的数据库均没有在训练、验证数据集中出现过。

>训练集文件会被按照4:1的比例拆分为训练集与验证集，测试集文件则对应测试集。

一般来说，在比较复杂的场景中，微调最少需要**1000+条优质的训练集数据**。构建数据集时，请确认以下几点：
*   **数据质量**：确保数据集的准确性和相关性，避免模糊和错误内容。
    
*   **数据多样性**：覆盖任务的所有关键方面和潜在变化，包括不同场景、语境和专业术语。
    
*   **平衡性**：如果任务涉及多种类别场景，确保各类别样本均衡，防止模型偏向于某一类。
    
*   **持续迭代**：微调是一个迭代过程，根据模型在验证集上的表现反馈，不断优化和扩大数据集。

如果微调后的模型评测结果不佳，最简单的改进方法是收集更多数据进行训练。
>由于本教程场景较为简单，为了演示方便并减少微调时间，本教程准备了550条左右的训练集数据。

如果你在进行模型微调时缺乏数据，建议你使用使用知识库检索来增强模型能力。
> 在很多复杂的业务场景，可以综合采用模型调优和知识库检索结合的技术方案。

你也可以采用以下策略扩充数据集：

1.  让能力更强的大模型模拟生成特定业务/场景的相关内容，辅助你生成更多可用于微调的数据。
    
2.  通过应用场景收集、网络爬虫、社交媒体和在线论坛、公开数据集、合作伙伴与行业资源、用户贡献等方式获取更多数据。

### 2.3. 获取基础大模型
你打算使用NL2SQL数据集对[qwen2.5-1.5b-instruct](https://modelscope.cn/models/Qwen/Qwen2.5-1.5B-Instruct)模型进行微调，首先你需要将模型文件下载到你的计算环境。你可以运行下边的命令来将模型下载到model文件夹中。

In [ ]:
!mkdir ./model
!modelscope download --model qwen/Qwen2.5-1.5B-Instruct --local_dir './model'

打开model文件夹，就可以看到Qwen2.5-1.5B-Instruct的模型文件了。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01AYvSIJ1kBSjIkeLcm_!!6000000004645-0-tps-604-688.jpg" alt="解压包文件" width="300px">

### 2.4. 使用swift框架进行微调
本课程使用[ms-swift框架](https://github.com/modelscope/ms-swift/tree/main) （ModelScope Scalable lightWeight Infrastructure for Fine-Tuning）作为微调框架，它是ModelScope社区专门为模型训练开发的开源框架，该框架支持350+ LLM和90+ MLLM（多模态大模型）的训练（预训练、微调、对齐）、推理、评测和部署。你可以直接将swift框架应用到自己的生产环境中，实现从模型的训练、评测、部署的完整链路。

#### 2.4.1. 使用swift框架进行模型推理
swift不仅可以用于模型微调，也可以进行模型的推理。你可以运行以下代码，测试微调前的大模型在NL2SQL任务上的表现。query参数用于指定输入的问题。
> 本步骤主要介绍使用swift框架进行模型推理的方法。下一步骤会介绍使用swift框架进行模型评测，可以帮助你对比模型微调前后的指标差别。

In [ ]:
from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType,
    get_default_template_type, inference_stream
)
import torch

def get_model_response(query):
    model_type = ModelType.qwen2_5_1_5b_instruct
    # 指定模型文件路径
    model_id_or_path = "./model"
    template_type = get_default_template_type(model_type)
    print(f'template_type: {template_type}')
    kwargs = {}
    # 通过设置cuda:0 使得模型加载到GPU上
    model, tokenizer = get_model_tokenizer(model_type, torch.float32, model_id_or_path=model_id_or_path, model_kwargs={'device_map': 'cuda:0'}, **kwargs)
    # 通过max_new_tokens指定大模型最多可以生成的token数
    model.generation_config.max_new_tokens = 128
    template = get_template(template_type, tokenizer)
    # 开启模型推理
    response, history = inference(model, template, query)
    print("输入：----------------------------------------------")
    print(query)
    print('----------------------------------------------')
    print("输出：\n", response)
    print("----------------------------------------------")

    # 在推理结束后清理内存和释放资源
    del model,template,tokenizer;import gc;_=gc.collect()

# 进行测试
query = "#背景#\n\
        数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'street address', 'text'], [6, 'postal code', 'text'], [6, 'city', 'text'], [6, 'state province', 'text'], [6, 'country id', 'text']], 'foreign_keys': [[5, 1], [20, 10], [24, 6], [28, 10], [29, 6], [25, 14], [35, 3]], 'primary_keys': [1, 3, 6, 10, 14, 25, 30], 'table_names': ['regions', 'countries', 'departments', 'jobs', 'employees', 'job history', 'locations']}\
        \n#受众#\n\
        Mysql数据库\n\
        #输出#\n\
        只输出SQL查询语句\n\
        #目的#\n\
        将问题\"What is all the information about employees who have never had a job in the past?\"转化为SQL查询语句'"
get_model_response(query)

#### 2.4.2 使用swift框架进行模型的评测
对模型进行微调是为了提高模型特定领域的能力，因此有必要在微调步骤的前后分别评测模型的表现，以此来检验微调是否带来了效果的提升。swift框架提供了模型评测的工具，评测的维度包含以下指标：
> 如果你无法理解这些概念也没有关系，你可以暂时理解为：这些分数越接近1，说明生成SQL语句有用部分越多，模型表现越好。

1.  **ROUGE Scores (Recall-Oriented Understudy for Gisting Evaluation)**:
    
    *   **rouge-{n}-(r/p/f)**: 这是基于词级别的n-gram匹配度量。n有1、2、l三个取值，如果n=1，表示统计单个词汇的重合情况；如果n=2，表示统计两个词汇组成的短语的重合情况；如果n=l，表示统计模型输出与测试用例（真实答案）中最长公共子序列的重合情况。
        
        *   **rouge-{n}-r**: 召回率（Recall），用于计算模型输出正确的n-gram数量除以真实答案中总的n-gram数量。
            
        *   **rouge-{n}-p**: 精确率（Precision），用于计算模型输出正确的n-gram数量除以模型总共输出的n-gram数量
            
        *   **rouge-{n}-f**: F1分数，是召回率和精确率的调和平均值，综合考量了两者的表现。
            

> 相关文章：ROUGE: A Package for Automatic Evaluation of Summaries (Chin-Yew Lin, 2004)

2.  **BLEU Scores (Bilingual evaluation understudy)**:
    
    *   **bleu-1/2/3/4**: 这些是基于n-gram的匹配度量，用于评估机器翻译或其他文本生成任务的质量。
        
        *   **bleu-1**: 基于单个词的匹配情况。
            
        *   **bleu-2**: 基于双词（bigram）的匹配情况。
            
        *   **bleu-3**: 基于三词（trigram）的匹配情况。
            
        *   **bleu-4**: 基于四词（4-gram）的匹配情况。
            

> 相关文章：BLEU: a Method for Automatic Evaluation of Machine Translation (Kishore Papineni, 2002)

测试集文件约有124条，每条数据遵循以下格式。

```json
{
    "history": ["表示对话历史"],
    "query": "融合了背景信息与输入问题",
    "response":  "query对应的真实答案"
}
```

> 评测模式为：general\_qa。相关信息请参见：[问答题格式数据集构建方法](https://evalscope.readthedocs.io/zh-cn/latest/advanced_guides/custom_dataset.html#qa)。
本微调教程没有涉及多轮对话的数据，因此history字段为`[]`。

通过运行以下命令来对模型进行评测，其中custom_eval_config参数的`resources/2_4/data/config_eval.json`文件通过dataset字段传入测试集文件的路径；`name`用于指定评测项名称，可以按需指定；`pattern`固定为`general_qa`即可。

<img src="https://img.alicdn.com/imgextra/i3/O1CN01MMbyoo1Stjw3Gjsr6_!!6000000002305-0-tps-1822-312.jpg" alt="评测文件截图" width="600px">

>评测完成需要等待大约3分钟的时间。


In [ ]:
!mkdir -p eval_outputs
!CUDA_VISIBLE_DEVICES=0 swift eval \
--model_id_or_path './model' \
--model_type 'qwen2_5-1_5b-instruct' \
--eval_dataset no \
--custom_eval_config 'resources/2_4/data/config_eval.json' \
--max_length -1 \
--system '' \
--infer_backend 'pt' \
--name 'pre_train_evaluation' \
--eval_output_dir './eval_outputs'

评测功能（swift eval）的完整参数列表请参见：[eval 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#eval)

虽然打印出了很多日志，但是你只需要关注倒数第三个`[INFO:swift]`对应的信息即可，可以看到评测结果：

```shell
{'test_data': {'rouge-1-r': 0.5137725087769407, 'rouge-1-p': 0.38776006330376156, 'rouge-1-f': 0.4308353617316975, 'rouge-2-r': 0.19387411758529038, 'rouge-2-p': 0.1451354381995863, 'rouge-2-f': 0.16236484129592232, 'rouge-l-r': 0.4733464478275048, 'rouge-l-p': 0.35809512534527843, 'rouge-l-f': 0.39239557904018646, 'bleu-1': 0.37097098165698, 'bleu-2': 0.17518403231621377, 'bleu-3': 0.10472123589582236, 'bleu-4': 0.04966604971853578}}
```

#### 2.4.3 使用swift框架进行模型的微调
在完成了模型的评测后，我们就可以开始对模型进行微调了。swift提供了操作简单的微调工具，你只需要传入训练集、模型路径等参数。swift框架在微调时默认使用LoRA方法，在命令中不需要额外声明。

>微调完成大约需要8分钟左右。

In [ ]:
!CUDA_VISIBLE_DEVICES=0 swift sft \
--dataset 'resources/2_4/data/train.jsonl' \
--learning_rate '1e-4' \
--eval_steps '10' \
--batch_size '4' \
--model_type 'qwen2_5-1_5b-instruct' \
--max_length 2048 \
--model_id_or_path './model' \
--num_train_epochs 3

微调（swift sft）完整参数列表请参见：[sft 命令行参数](https://swift.readthedocs.io/zh-cn/latest/Instruction/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.html#sft-%E5%8F%82%E6%95%B0) 

在微调时，并非所有训练样本都会直接参与到模型参数的训练中，swift框架会以4:1的比例将训练样本划分为训练集与验证集。其中训练集会直接参与到模型参数的训练；验证集虽然不直接参与训练过程，但在模型训练时提供了实时评测的效果，通过设置`--eval_steps`参数指定每训练多少steps，进行评测与保存当前模型的checkpoint，这样swift框架可以保存在验证集上获得最佳评测效果的checkpoint，作为最佳模型（即日志中打印的best_model_checkpoint）。

>checkpoint（检查点）是指在训练过程中定期保存模型当前状态的机制，可以帮助模型恢复训练、进行评估、防止过拟合等。这些检查点包括模型的参数、优化器状态以及其他可能影响训练过程的信息。

在打印的日志中包含两个重要指标：训练损失（loss）和验证损失（eval_loss）。

完整的loss和eval_loss信息保存在文件`output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxx/logging.jsonl`中。

*   **如果你观察到loss在减少，eval_loss也在减少**，说明模型需要继续训练。你可以增加`--num_train_epochs`的值来增加模型的训练深度。  
<br><i style="color: #999"> 可以将当前的模型类比为一个处在上升期的运动员，通过不断的练习即可稳步提升比赛成绩。 </i><br>
    
*   **如果你观察到loss在减少，eval_loss却在增加，**说明模型已经处于**过拟合**状态，过度拟合了训练数据中的潜在模式。在LoRA微调中，可以通过增加`--lora_dropout`的值，抑制模型的**过拟合**倾向。
<br><i style="color: #999"> 可以将当前的模型类比为一个只会背题库的学生，无法掌握题目背后的机理。 </i><br>
    
*   **如果你观察到loss在增加，eval_loss也增加，**（不常见）说明模型训练需要立刻停止。常见的原因有：
    *   学习率设置过大，导致模型参数剧烈震荡。
        
    *   训练数据有问题，例如模型输入和预期输出不存在任何逻辑关系。
        
    *   抑制模型过拟合的参数设置过大，导致模型无法学习新知识。

你可以在output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/images文件夹查看训练过程中的loss、eval_loss等指标的变化情况（图片格式）。

你可以运行以下代码，测试微调后模型的输出情况。<br>
运行以下代码前请修改ckpt_dir为在验证集表现最佳的微调模型路径。微调后一般会保存两个checkpoint文件，分别是best_model_checkpoint（在验证集表现最佳的微调模型）与last_model_checkpoint（最后一次保存的checkpoint）。

best_model_checkpoint的路径在上文运行记录的倒数第三个[INFO:swift]日志中，我们推荐你传入best_model_checkpoint的路径。

In [ ]:
from swift.tuners import Swift

# 请在运行前修改ckpt_dir到best_model_checkpoint对应的路径
ckpt_dir = 'output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx'

# 你可以根据你的需要修改query（模型输入）
query = "#背景#\n\
    数据库信息:{'column_names': [[-1, '*', 'text'], [0, 'region id', 'number'], [0, 'region name', 'text'], [1, 'country id', 'text'], [1, 'country name', 'text'], [1, 'region id', 'number'], [2, 'department id', 'number'], [2, 'department name', 'text'], [2, 'manager id', 'number'], [2, 'location id', 'number'], [3, 'job id', 'text'], [3, 'job title', 'text'], [3, 'min salary', 'number'], [3, 'max salary', 'number'], [4, 'employee id', 'number'], [4, 'first name', 'text'], [4, 'last name', 'text'], [4, 'email', 'text'], [4, 'phone number', 'text'], [4, 'hire date', 'time'], [4, 'job id', 'text'], [4, 'salary', 'number'], [4, 'commission pct', 'number'], [4, 'manager id', 'number'], [4, 'department id', 'number'], [5, 'employee id', 'number'], [5, 'start date', 'time'], [5, 'end date', 'time'], [5, 'job id', 'text'], [5, 'department id', 'number'], [6, 'location id', 'number'], [6, 'street address', 'text'], [6, 'postal code', 'text'], [6, 'city', 'text'], [6, 'state province', 'text'], [6, 'country id', 'text']], 'foreign_keys': [[5, 1], [20, 10], [24, 6], [28, 10], [29, 6], [25, 14], [35, 3]], 'primary_keys': [1, 3, 6, 10, 14, 25, 30], 'table_names': ['regions', 'countries', 'departments', 'jobs', 'employees', 'job history', 'locations']}\
    \n#受众#\n\
    Mysql数据库\n\
    #输出#\n\
    只输出SQL查询语句\n\
    #目的#\n\
    将问题\"What is all the information about employees who have never had a job in the past?\"转化为SQL查询语句'"

model_type = ModelType.qwen2_5_1_5b_instruct
template_type = get_default_template_type(model_type)
model_id_or_path = "./model"
model, tokenizer = get_model_tokenizer(model_type, torch.float32, model_id_or_path=model_id_or_path, model_kwargs={'device_map': 'cuda:0'})
model.generation_config.max_new_tokens = 128
# 通过ckpt_dir传入微调模型
model = Swift.from_pretrained(model, ckpt_dir, inference_mode=True)
import gc;_=gc.collect();
template = get_template(template_type, tokenizer)
response, history = inference(model, template, query)
print("输入：----------------------------------------------")
print(query)
print('----------------------------------------------')
print("输出：\n", response)
print("----------------------------------------------")
del model,template,tokenizer;import gc;_=gc.collect();

可以看到，相比于微调前的模型，微调后的模型输出：`SELECT * FROM employees WHERE employee_id NOT IN (SELECT DISTINCT employee_id FROM job_history)`不再有前后缀等多余信息，并且结尾处也没有了`；`这符合我们在`train.jsonl`文件中指定response的风格。

#### 2.4.4. 使用swift框架融合模型
模型微调训练完成后，有两种方式可以使用微调后的模型：

1. 将基础模型与微调模型融合为一个模型后调用

    我们使用的Qwen2.5-1.5B-Instruct模型有15亿参数，微调模型有2百万参数，由于融合阶段会将微调模型参数矩阵叠加到基础模型参数矩阵，因此微调后的模型仍然有15亿参数，这种方式适合做整合好的模型版本发布。

2. 在调用时动态加载微调模型。

    微调模型有2百万参数，假设微调模型参数矩阵只占100兆字节存储空间，这个大小非常便于做增量发布和传播，这也是工程上常用的方法。需要注意的是，用哪个基础模型微调，在加载时就需要指定使用哪个基础模型。

这里我们介绍第一种方法：融合“微调参数矩阵”与“基础模型参数矩阵，将微调后的模型参数存储成一个完整的参数矩阵。

通过swift export方法，传入微调模型的路径（建议传入best_model_checkpoint），即可得到融合后的模型。

In [ ]:
!CUDA_VISIBLE_DEVICES=0 swift export \
--ckpt_dir 'output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx' \
--merge_lora true

倒数第二行日志展示了融合后模型的路径。你可以使用前文介绍的swift评测方法来对融合后的模型进行评测，将融合后的模型路径传入`model_id_or_path`参数中。

In [ ]:
!mkdir -p eval_outputs
!CUDA_VISIBLE_DEVICES=0 swift eval \
--model_id_or_path './output/qwen2_5-1_5b-instruct/v2-202xxx-xxxxxx/checkpoint-xxx-merged' \
--model_type 'qwen2_5-1_5b-instruct' \
--eval_dataset no \
--custom_eval_config 'resources/2_4/data/config_eval.json' \
--max_length -1 \
--system '' \
--infer_backend 'pt' \
--name 'pre_train_evaluation' \
--eval_output_dir './eval_outputs'

倒数第三个`[INFO:swift]`对应的信息是评测的结果，我们可以看到评测结果：

微调前：
```shell
{'test_data': {'rouge-1-r': 0.5137725087769407, 'rouge-1-p': 0.38776006330376156, 'rouge-1-f': 0.4308353617316975, 'rouge-2-r': 0.19387411758529038, 'rouge-2-p': 0.1451354381995863, 'rouge-2-f': 0.16236484129592232, 'rouge-l-r': 0.4733464478275048, 'rouge-l-p': 0.35809512534527843, 'rouge-l-f': 0.39239557904018646, 'bleu-1': 0.37097098165698, 'bleu-2': 0.17518403231621377, 'bleu-3': 0.10472123589582236, 'bleu-4': 0.04966604971853578}}
```

微调后：
```shell
{'test_data': {'rouge-1-r': 0.6918848406176754, 'rouge-1-p': 0.6496502831544233, 'rouge-1-f': 0.6531929692672186, 'rouge-2-r': 0.4592924596772488, 'rouge-2-p': 0.43600353982252626, 'rouge-2-f': 0.43876407584399113, 'rouge-l-r': 0.6694186618986864, 'rouge-l-p': 0.6252798038344263, 'rouge-l-f': 0.6271772601048794, 'bleu-1': 0.6214933432699985, 'bleu-2': 0.4732749320454375, 'bleu-3': 0.37427255262995696, 'bleu-4': 0.27419057449549805}}
```

可以看到，各项分数都有了较大提升，证明经过微调后的模型NL2SQL的能力得到了大幅度的加强。

## ✅ 本节小结
通过本节内容的学习。你已经学会了如何使用LoRA微调方法来提升大模型的NL2SQL能力。

虽然在本教程中你可以使用我们提供的NL2SQL数据集，免费体验GPU算力资源进行微调，但在实际生产中，微调往往并不简单。高昂的算力成本与对数据质量的严苛要求都是你在微调前需要考虑的因素。


在开始微调任务之前，我们建议你关注以下方面：
1. 先优化提示词<br>
我们建议你先优化提示词，如果提示词的优化已无法继续提升效果，再考虑模型微调。
2. 先用少量数据进行测试
先用少量数据测试（比如100条以内），观察效果是否有所提升。如果效果提升，说明数据集质量较高，你可以以此数据集为标准继续扩充数据；如果效果没有提升，说明数据集质量较低，你需要排查一下质量较低的原因。
3. 明确业务需求匹配度<br>
首先，你需要清楚业务的具体需求，明确微调要解决的具体问题，并评估当前模型是否已能满足大部分需求，如果是，则可能不需要微调。确定是否有明确的业务指标来衡量微调前后的效果，例如NL2SQL的查询成功率。
4. 尝试更大的模型<br>
你可以考虑使用参数更大的模型，因为它们经过更多数据和训练轮次，在通用领域和特定领域都可能带来显著的性能提升。
即使有使用成本和延迟限制，也建议进行实验或性能验证。更大的模型在微调时可以作为“老师”，传授知识给较小的“学生”模型。
5. 使用提示词工程和插件调用<br>
这种方法比对模型进行微调更灵活、成本更低，并且这些前期工作可以在构建微调数据集时复用。
6. 评估资源和技术可行性<br>
微调过程需要计算资源和时间成本，包括GPU资源（训练时需要）、存储空间（存储模型时需要）和专家人力成本（构建数据集时需要）。评估项目预算和资源是否允许进行有效微调，同时确保团队具备所需的技术能力，或寻求合作伙伴提供支持（你可以直接使用[百炼提供的0代码微调能力](https://help.aliyun.com/zh/model-studio/user-guide/using-fine-tuning-on-console)）。
7. 确保充足的数据<br>
虽然微调相比于预训练阶段对数据要求降低，但仍需准备足够的精确标注数据集，最少需要1000条数据才能获得良好效果。
8. 关注合规与隐私<br>
确保使用的数据符合法律法规要求，处理个人数据时遵循隐私保护原则。

微调前的这些准备工作将帮助我们更好地实现微调目标，提升业务效果。

由于2.8中的内容涉及到模型部署，也需要GPU示例，因此你可以继续留在这个GPU实例中学习2.8的课程内容。在2.8的教程中，你将了解如何将本地模型（是否经过微调都可以）部署到阿里云ECS或PAI上。如果你要部署微调模型，需要部署参数矩阵融合后生成的完整参数矩阵（后缀带merged），格式为：output/qwen2_5-1_5b-instruct/v2-202xxxxx-xxxxxx/checkpoint-xxx-merged。

## 🔥 课后小测验

【单选题】以下关于 LoRA 的描述，哪一项是错误的？（ ）

A. LoRA 可以有效降低微调大型语言模型的成本。

B. LoRA 会修改预训练模型的原始权重。

C. LoRA 的实现相对简单，易于集成。

D. LoRA 微调的结果可以方便地回退。

答案：B

解析： LoRA 并不直接修改原始权重，而是通过添加低秩矩阵间接影响模型行为。 这使得回退操作变得简单，只需移除添加的低秩矩阵即可。

<br>

【多选题】你正在使用 Swift 微调一个 Qwen 模型，发现模型在验证集上的 loss 出现了明显的上升趋势，以下哪些操作可以帮助你缓解或解决这个问题？( )

A. 增大 --learning_rate

B. 减小 --learning_rate

C. 增大 --lora_dropout

D. 减小 --lora_dropout

E. 增大 --num_train_epochs

F. 减小 --num_train_epochs

答案: B, C, F

解释:

- learning_rate: 学习率过大会导致模型训练速度快，但可能在最优解附近震荡，甚至无法收敛，导致loss波动，看起来像是过拟合。当然这与真正的过拟合不同。
- lora_dropout: 增大 dropout 率可以增强模型的泛化能力，抑制过拟合。
- num_train_epochs: 过拟合也可能是训练次数过多导致的，减少训练次数可以避免模型过度学习训练数据。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。